In [1]:
import pandas as pd
from fastparquet import ParquetFile
import duckdb
from bs4 import BeautifulSoup
import requests

In [2]:
p1 = ParquetFile("first_half.parquet").to_pandas()

In [64]:
p2 = ParquetFile("second_half.parquet").to_pandas()

In [65]:
p1 = p1.dropna(subset="item").drop(columns=["id"])
p2 = p2.rename(columns={"id" : "item"})
pd.concat([p1,p2]).drop_duplicates().to_parquet("first_half.parquet")

In [12]:
item_df = pd.read_csv("./item_rarities.csv")

In [20]:
item_rarity_map = {
    "Covert": "red",
    "Classified" : "pink",
    "Restricted" : "purple",
    "Mil-spec" : "blue",
    "Industrial": "light-blue",
    "Consumer" : "gray"
}

item_df["Color Rarity"] = item_df["Rarity"].map(item_rarity_map)

In [67]:
graffiti_df = p1[p1["item"].str.contains("Graffiti", na=False)]
graffiti_df.to_parquet("graffiti.parquet")

In [ ]:
sticker_df = p1[p1["item"].str.contains("Sticker", na=False)]

In [3]:
url = "https://raw.githubusercontent.com/ByMykel/CSGO-API/main/public/api/en"


In [5]:
res = requests.get(url + "/stickers.json")
data = res.json()

sticker_data_df = pd.json_normalize(data, sep="_")

want = {
    "id": "id",
    "name": "name",
    "type": "type",
    "effect": "effect",
    "market_hash_name": "market_hash_name",
    "rarity_name": "rarity",
    "rarity_color": "rarity_color",
    "tournament_id": "tournament_id",
    "tournament_name": "tournament",
    "team_id": "team_id",
    "team_tag": "team_tag",
    "team_name": "team",
    "image": "image_url",
}

out = (sticker_data_df.rename(columns=want).reindex(columns=want.values(), fill_value=pd.NA))

out = out.drop(columns=["id"]).rename(columns={"name": "item"})
duckdb.query("SELECT DISTINCT rarity_color FROM out")

┌──────────────┐
│ rarity_color │
│   varchar    │
├──────────────┤
│ #8847ff      │
│ #ded6cc      │
│ #eb4b4b      │
│ #e4ae39      │
│ #d32ce6      │
│ #4b69ff      │
└──────────────┘

In [ ]:
sticker_color_map = {
    "#e4ae39": "gold",
    "#eb4b4b" : "red",
    "#8847ff" : "purple",
    "#d32ce6" : "pink",
    "#4b69ff" : "blue",
    "#ded6cc" : "gray"
}


out["color"] = out["rarity_color"].map(sticker_color_map)
out = out.drop(columns=["rarity_color"])
out = out.rename(columns={"color": "rarity_color"}).copy()

In [10]:
out.head()

,item,type,effect,market_hash_name,rarity,rarity_color,tournament_id,tournament,team_id,team_tag,team,image_url,color
0,Sticker | Shooter,Event,Other,None,Default,#ded6cc,1.0,2013 DreamHack Winter,NaN,NaN,NaN,https://cdn.steamstatic.com/apps/730/icons/eco...,gray
1,Sticker | Shooter (Foil),Event,Foil,None,Default,#ded6cc,1.0,2013 DreamHack Winter,NaN,NaN,NaN,https://cdn.steamstatic.com/apps/730/icons/eco...,gray
2,Sticker | Shooter Close,Event,Other,None,Default,#ded6cc,1.0,2013 DreamHack Winter,NaN,NaN,NaN,https://cdn.steamstatic.com/apps/730/icons/eco...,gray
3,Sticker | Shooter Close (Foil),Event,Foil,None,Default,#ded6cc,1.0,2013 DreamHack Winter,NaN,NaN,NaN,https://cdn.steamstatic.com/apps/730/icons/eco...,gray
4,Sticker | Blue Snowflake,Event,Other,None,Default,#ded6cc,1.0,2013 DreamHack Winter,NaN,NaN,NaN,https://cdn.steamstatic.com/apps/730/icons/eco...,gray
